# Machine Learning Final Project

Joey Couse and Abraham Eaton <br>
11-November-2019 <br>

In [1]:
# using JuMP, Gurobi, Statistics, DataFrames, LinearAlgebra, StatsBase, CSV, Random, Suppressor

In [2]:
# Data=CSV.read("C:\\Users\\Abraham\\Dropbox (MIT)\\New folder\\housing.csv",header=true);
#Data=convert(Matrix,Data);
#Data=Data[shuffle(1:end),:];

In [3]:
#Add something to handle categorical variables

In [4]:
# test_obs_count=round(Int,0.1*size(Data,1));
# Test_Data = Data[1:test_obs_count,:];
# Data=Data[(test_obs_count+1):end,:];

In [5]:


# n=size(Data,1);
# print("Number of observations: ")
# println(n);
# m=size(Data,2);
# println();

# X=Data[:,1:(m-1)];
# Y=Data[:,m];
# test_X=Test_Data[:,1:(m-1)];
# test_Y=Test_Data[:,m];


# m=m-1;
# print("Number of Features: ")
# println(m);
# println();
# print("Percent of data in validation set: ")
# ρ=0.25;
# println(ρ);

Number of observations: 455

Number of Features: 13

Percent of data in validation set: 0.25


In [33]:
# Balanced = Model(solver=GurobiSolver(TimeLimit=90));

# @variable(Balanced, z[1:n], Bin);
# @constraint(Balanced, sum(z[:])==floor(ρ*n));

# @objective(Balanced, Min, sum(1/floor(ρ*n)*sum(z[i]*X[i,j] for i = 1:n) - 
#     1/ceil((1-ρ)*n)*sum((1-z[i])*X[i,j] for i = 1:n) for j = 1:m));

# solve(Balanced);


Academic license - for non-commercial use only
Optimize a model with 1 rows, 455 columns and 455 nonzeros
Variable types: 0 continuous, 455 integer (455 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective 1224.5970916
Presolve time: 0.01s
Presolved: 1 rows, 455 columns, 455 nonzeros
Variable types: 0 continuous, 455 integer (455 binary)

Root relaxation: objective 9.844904e+02, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     984.4904198  984.49042  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 984.49 1224.6 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.844904197769e

In [35]:
# sum(getvalue(z));

113.0

In [90]:
# k= (1-ρ)*n; #number of observations in training set
# λ= 1; #regularization constant

# Stable = Model(solver=GurobiSolver(TimeLimit=90));

# @variable(Stable, B[1:m]);
# @variable(Stable, θ);
# @variable(Stable, μ[1:n] >=0);

# @constraint(Stable, [i=1:n], θ+μ[i] >= Y[i]-sum(X[i,:].*B));
# @constraint(Stable, [i=1:n], θ+μ[i] >= -Y[i]+sum(X[i,:].*B));

# @objective(Stable, Min, k*θ+sum(μ[i] for i = 1:n)+λ*sum(B.^2));

# solve(Stable);

In [91]:
# Betas=getvalue(B);

In [92]:
# MSE=mean((sum(Betas.*test_X[i,:]) .- test_Y[i]).^2 for i = 1:size(test_X,1))

22.22200390597508

In [1]:
using JuMP, Gurobi, Statistics, DataFrames, LinearAlgebra, StatsBase, CSV, Random, Suppressor

In [2]:
SVB = function(Data,λ0=0,granularity=1,runs=10,print_results=true,seed=rand(1:2000019),percent_validation=0.25,Y_col_last=true,test_percentage=0.1)
    
    if print_results==true
        print("Seed: ")
        println(seed);
        println();
        println("-------------------------------------------------------------")
        println();
    end
    
    function LASSO_Regression(X,Y,tun)
        features=size(X,2);
        obs=size(Y,1);
        #M=10000;
        #println("debug -2");
        LASSO = Model(solver=GurobiSolver(TimeLimit=30));
        @variable(LASSO, Bn[1:features]);
        @variable(LASSO, obj >= 0);
        @constraint(LASSO, obj >= sum((Y-X*Bn).^2)+tun*sum(Bn.^2))
        @objective(LASSO,Min,obj);
        
        #redirect_stdout((()->solve(LASSO)),open("C:\\Users\\Abraham\\Documents\\null","w"))
        solve(LASSO);
        #println("debug -1");
        Bs=getvalue(Bn);
        return Bs
    end
    
    
    
    ρ=percent_validation;
    
    ##
    ##
    ##
    ## Data Pre-Processing
    ##
    ##
    ##
    
    Data=convert(Matrix,Data);
    Random.seed!(seed);
    Data=Data[shuffle(1:end),:];
    
    
    test_obs_count=round(Int,test_percentage*size(Data,1));
    Test_Data = Data[1:test_obs_count,:];
    Data=Data[(test_obs_count+1):end,:];

    
    
    n=size(Data,1);
    m=size(Data,2);
    
    if (Y_col_last == true)
        X=Data[:,1:(m-1)];
        Y=Data[:,m];
        test_X=Test_Data[:,1:(m-1)];
        test_Y=Test_Data[:,m];
    else
        X=Data[:,2:(m)];
        Y=Data[:,1];
        test_X=Test_Data[:,2:(m)];
        test_Y=Test_Data[:,1];
    end
    m=m-1;
    #ρ=0.25;
    
    ##
    ##
    ##
    ##
    ## Balanced Regression
    ##
    ##
    ##
    ##
    if print_results==true
        println("Balanced Regression Results:");
        println();
    end
    
    Balanced = Model(solver=GurobiSolver(TimeLimit=5));

    @variable(Balanced, z[1:n], Bin);
    @constraint(Balanced, sum(z[:])==floor(ρ*n));
    @variable(Balanced, t >= 0);
#     @constraint(Balanced, t>=
#         sum(1/floor(ρ*n)*sum(z[i]*X[i,j] for i = 1:n) - 
#         1/ceil((1-ρ)*n)*sum((1-z[i])*X[i,j] for i = 1:n) 
#             for j = 1:m)^2 +
#         sum(1/floor(ρ*n)*sum(z[i]*Y[i] for i = 1:n) - 
#         1/ceil((1-ρ)*n)*sum((1-z[i])*Y[i] for i = 1:n))^2);
    @constraint(Balanced, t>=sum(
            1/floor(ρ*n)*sum(z[i]*X[i,j] for i = 1:n) - 
        1/ceil((1-ρ)*n)*sum((1-z[i])*X[i,j] for i = 1:n) 
            for j = 1:m)+
        sum(
            1/floor(ρ*n)*sum(z[i]*Y[i] for i = 1:n) - 
        1/ceil((1-ρ)*n)*sum((1-z[i])*Y[i] for i = 1:n)));
    @constraint(Balanced, t>=sum(
            1/floor(ρ*n)*sum(z[i]*X[i,j] for i = 1:n) - 
        1/ceil((1-ρ)*n)*sum((1-z[i])*X[i,j] for i = 1:n) 
            for j = 1:m)+
        sum(
            -1/floor(ρ*n)*sum(z[i]*Y[i] for i = 1:n) + 
        1/ceil((1-ρ)*n)*sum((1-z[i])*Y[i] for i = 1:n)));
    @constraint(Balanced, t>=sum(
            -1/floor(ρ*n)*sum(z[i]*X[i,j] for i = 1:n) + 
        1/ceil((1-ρ)*n)*sum((1-z[i])*X[i,j] for i = 1:n) 
            for j = 1:m)+
        sum(
            1/floor(ρ*n)*sum(z[i]*Y[i] for i = 1:n) - 
        1/ceil((1-ρ)*n)*sum((1-z[i])*Y[i] for i = 1:n)));
    @constraint(Balanced, t>=sum(
            -1/floor(ρ*n)*sum(z[i]*X[i,j] for i = 1:n) + 
        1/ceil((1-ρ)*n)*sum((1-z[i])*X[i,j] for i = 1:n) 
            for j = 1:m)+
        sum(
            -1/floor(ρ*n)*sum(z[i]*Y[i] for i = 1:n) + 
        1/ceil((1-ρ)*n)*sum((1-z[i])*Y[i] for i = 1:n)));
        
    @objective(Balanced, Min, t);

    #redirect_stdout((()->solve(Balanced)),open("C:\\Users\\Abraham\\Documents\\null","w"));
    solve(Balanced);
    
    if print_results == true
        println("The objective value of the balancing is: ");
        println(getobjectivevalue(Balanced));
        println();
    end
    
    ###
    ###
    ###
    ###
    ### Do Lasso stuff here
    ###
    ###
    ###
    ###
    
    Betas_Mat=zeros(runs,m);
    λval=zeros(runs);
    
    Is_Validation=round.(Int,getvalue(z));
    
    sum(Is_Validation);
    
    for i = 1:runs
        λ = (i-1)*granularity+λ0;
        Betas_Mat[i,:]=LASSO_Regression(X[Is_Validation.==0,:],Y[Is_Validation.==0,:],λ);
        λval[i]=λ;
    end
    #println("Debug 0");
    
    valid_X=X[Is_Validation.==1,:];
    valid_Y=Y[Is_Validation.==1,:];
    
    MSE_Best_Balanced=mean((sum(Betas_Mat[1,:].*valid_X[i,:]) .- valid_Y[i]).^2 for i = 1:size(valid_X,1));
    λbest=λval[1];
    β_Best_Balanced=Betas_Mat[1,:];
    
    
    for j = 1:runs
        MSE = mean((sum(Betas_Mat[j,:].*valid_X[i,:]) .- valid_Y[i]).^2 for i = 1:size(valid_X,1));
        if (MSE <= MSE_Best_Balanced)
            MSE_Best_Balanced = MSE;
            λbest=λval[j];
            β_Best_Balanced=Betas_Mat[j,:];
        end
    end
    
    if print_results==true
        println("Best Balanced Lambda:");
        println(λbest);
        println();
        println("Best Balanced Validation MSE: ");
        println(MSE_Best_Balanced);
        println();
    end
    
    β_Best_Balanced=LASSO_Regression(X,Y,λbest);
    #println("Debug 1")
    
    MSE_Best_Balanced=mean((sum(β_Best_Balanced.*test_X[i,:]) .- test_Y[i]).^2 for i = 1:size(test_X,1));
    
    if print_results==true
        println("Test MSE for Balanced Regression:")
        println(MSE_Best_Balanced);
        println();
        println("-------------------------------------------------------------")
    end
    
    ###
    ###
    ###
    ###
    ### This is where Stable Regression starts
    ###
    ###
    ###
    ###
    
    train_obs_count= (1-ρ)*n; #number of observations in training set
    λ= 1; #regularization constant
    
    Stable_Regression = function(X,Y,k,λ)
        m=size(X,2);
        n=size(Y,1);
        
        Stable = Model(solver=GurobiSolver(TimeLimit=30));

        @variable(Stable, B[1:m]);
        @variable(Stable, θ);
        @variable(Stable, μ[1:n] >=0);

        @constraint(Stable, [i=1:n], θ+μ[i] >= Y[i]-sum(X[i,:].*B));
        @constraint(Stable, [i=1:n], θ+μ[i] >= -Y[i]+sum(X[i,:].*B));

        @objective(Stable, Min, k*θ+sum(μ[i] for i = 1:n)+λ*sum(B.^2));
        #redirect_stdout((()->solve(Stable)),open("C:\\Users\\Abraham\\Documents\\null","w"));
        solve(Stable);
        Betas_Stable=getvalue(B);
        Is_Train=Int.(getvalue(μ).>=0.0001);
        #Is_Train=getvalue(μ);
        return Betas_Stable, Is_Train
    end
    
    if print_results==true
        println("Stable Regression Results:");
        println();
    end
    
    Betas_Mat=zeros(runs,m);
    λval=zeros(runs);
    Is_Train_Mat=zeros(size(X,1),runs);

    #println("Debug 3")
    for i = 1:runs
        λ = (i-1)*granularity+λ0;
        res=Stable_Regression(X,Y,train_obs_count,λ);
        Betas_Mat[i,:]=res[1];
        Is_Train_Mat[:,i]=res[2];
        λval[i]=λ;
    end
    
    Stable_Valid_MSE=10000;
    λbest_Stable=-100;
    Best_Betas_Stable=zeros(size(Betas_Mat,2));
    
    for j = 1:runs
        valid_X=X[Is_Train_Mat[:,j].==0,:];
        valid_Y=Y[Is_Train_Mat[:,j].==0,:];
        MSE = mean((sum(Betas_Mat[j,:].*valid_X[i,:]) .- valid_Y[i]).^2 for i = 1:size(valid_X,1));
        
        if (MSE <= Stable_Valid_MSE)
            Stable_Valid_MSE=MSE;
            λbest_Stable=λval[j];
            Best_Betas_Stable=Betas_Mat[j,:];
        end
    end
    
    if print_results==true
        println("Best Stable Lambda: ");
        println(λbest_Stable);
        println();
        println("Best Stable Validation MSE: ");
        println(Stable_Valid_MSE);
        println();
    end
    
    MSE_Best_Stable=mean((sum(Best_Betas_Stable.*test_X[i,:]) .- test_Y[i]).^2 for i = 1:size(test_X,1));
    
    if print_results==true
        println("Test MSE for Stable Regression:")
        println(MSE_Best_Stable);
        println();
        println("-------------------------------------------------------------")
    end
    
    return MSE_Best_Balanced, MSE_Best_Stable
end

#3 (generic function with 9 methods)

In [3]:
Data=CSV.read("C:\\Users\\Abraham\\Dropbox (MIT)\\New folder\\housing.csv",header=true);

In [4]:
j=0;
k=0;
iterations=1000;

Housing_Results=zeros(iterations,2)

for i = 1:iterations
    comp = @suppress SVB(Data,0,1,10,false);
    
    Housing_Results[i,1]=comp[1];
    Housing_Results[i,2]=comp[2];
    
    if comp[1]<comp[2]
        j=j+1;
    end
    if comp[2]<comp[1]
        k=k+1;
    end
    if (i%10==0)
        print((i/iterations)*100);
        println("% completed ");
        print(j);
        print("-");
        println(k);
        println("--------------");
    end
end

1.0% completed 
4-6
--------------
2.0% completed 
12-8
--------------
3.0% completed 
16-14
--------------
4.0% completed 
23-17
--------------
5.0% completed 
27-23
--------------
6.0% completed 
34-26
--------------
7.000000000000001% completed 
40-30
--------------
8.0% completed 
46-34
--------------
9.0% completed 
55-35
--------------
10.0% completed 
61-39
--------------
11.0% completed 
68-42
--------------
12.0% completed 
72-48
--------------
13.0% completed 
80-50
--------------
14.000000000000002% completed 
84-56
--------------
15.0% completed 
88-62
--------------
16.0% completed 
94-66
--------------
17.0% completed 
101-69
--------------
18.0% completed 
109-71
--------------
19.0% completed 
115-75
--------------
20.0% completed 
122-78
--------------
21.0% completed 
130-80
--------------
22.0% completed 
134-86
--------------
23.0% completed 
138-92
--------------
24.0% completed 
146-94
--------------
25.0% completed 
153-97
--------------
26.0% completed 
159-101


In [5]:
println(j)
println(k)
println(j/(j+k))
println(j+k)

584
416
0.584
1000


In [6]:
## New Data Set
Data=CSV.read("C:\\Users\\Abraham\\Dropbox (MIT)\\New folder\\forestfires.csv",header=true);
Data=Data[:,[1,2,5,6,7,8,9,10,11,12,13]];

In [7]:
j=0;
k=0;
iterations=1000;

ForestFire_Results=zeros(iterations,2)

for i = 1:iterations
    comp = @suppress SVB(Data,0,1,10,false);
    
    ForestFire_Results[i,1]=comp[1];
    ForestFire_Results[i,2]=comp[2];
    
    if comp[1]<comp[2]
        j=j+1;
    end
    if comp[2]<comp[1]
        k=k+1;
    end
    if (i%10==0)
        print((i/iterations)*100);
        println("% completed ");
        print(j);
        print("-");
        println(k);
        println("--------------");
    end
end

1.0% completed 
4-6
--------------
2.0% completed 
8-12
--------------
3.0% completed 
12-18
--------------
4.0% completed 
16-24
--------------
5.0% completed 
21-29
--------------
6.0% completed 
25-35
--------------
7.000000000000001% completed 
29-41
--------------
8.0% completed 
34-46
--------------
9.0% completed 
39-51
--------------
10.0% completed 
43-57
--------------
11.0% completed 
48-62
--------------
12.0% completed 
55-65
--------------
13.0% completed 
58-72
--------------
14.000000000000002% completed 
63-77
--------------
15.0% completed 
68-82
--------------
16.0% completed 
74-86
--------------
17.0% completed 
79-91
--------------
18.0% completed 
85-95
--------------
19.0% completed 
91-99
--------------
20.0% completed 
94-106
--------------
21.0% completed 
96-114
--------------
22.0% completed 
102-118
--------------
23.0% completed 
107-123
--------------
24.0% completed 
114-126
--------------
25.0% completed 
117-133
--------------
26.0% completed 
121-139

In [8]:
println(j)
println(k)
println(j/(j+k))
println(j+k)

479
521
0.479
1000


In [9]:
#CSV.write("C:\\Users\\Abraham\\Dropbox (MIT)\\New folder\\Results.csv",DataFrame(hcat(Housing_Results,ForestFire_Results)),writeheader=false);

In [10]:
## New Data Set
Data=CSV.read("C:\\Users\\Abraham\\Dropbox (MIT)\\New folder\\Concrete_Data.csv",header=true);
#Data=Data[1:400,:];

In [11]:
j=0;
k=0;
iterations=1000;

Concrete_Results=zeros(iterations,2)

for i = 1:iterations
    comp = @suppress SVB(Data,0,1,10,false);
    
    Concrete_Results[i,1]=comp[1];
    Concrete_Results[i,2]=comp[2];
    
    if comp[1]<comp[2]
        j=j+1;
    end
    if comp[2]<comp[1]
        k=k+1;
    end
    if (i%10==0)
        print((i/iterations)*100);
        println("% completed ");
        print(j);
        print("-");
        println(k);
        println("--------------");
    end
end

1.0% completed 
4-6
--------------
2.0% completed 
8-12
--------------
3.0% completed 
13-17
--------------
4.0% completed 
17-23
--------------
5.0% completed 
25-25
--------------
6.0% completed 
31-29
--------------
7.000000000000001% completed 
38-32
--------------
8.0% completed 
47-33
--------------
9.0% completed 
53-37
--------------
10.0% completed 
61-39
--------------
11.0% completed 
65-45
--------------
12.0% completed 
73-47
--------------
13.0% completed 
81-49
--------------
14.000000000000002% completed 
85-55
--------------
15.0% completed 
91-59
--------------
16.0% completed 
98-62
--------------
17.0% completed 
105-65
--------------
18.0% completed 
114-66
--------------
19.0% completed 
122-68
--------------
20.0% completed 
128-72
--------------
21.0% completed 
133-77
--------------
22.0% completed 
138-82
--------------
23.0% completed 
147-83
--------------
24.0% completed 
154-86
--------------
25.0% completed 
160-90
--------------
26.0% completed 
167-93
-

In [12]:
println(j)
println(k)
println(j/(j+k))
println(j+k)

682
318
0.682
1000


In [13]:
## New Data Set
Data=CSV.read("C:\\Users\\Abraham\\Dropbox (MIT)\\New folder\\CompHard.csv",header=true);
Data=Data[:,3:9];

In [14]:
j=0;
k=0;
iterations=1000;

CompHard_Results=zeros(iterations,2)

for i = 1:iterations
    comp = @suppress SVB(Data,0,1,10,false);
    
    CompHard_Results[i,1]=comp[1];
    CompHard_Results[i,2]=comp[2];
    
    if comp[1]<comp[2]
        j=j+1;
    end
    if comp[2]<comp[1]
        k=k+1;
    end
    if (i%10==0)
        print((i/iterations)*100);
        println("% completed ");
        print(j);
        print("-");
        println(k);
        println("--------------");
    end
end

1.0% completed 
2-8
--------------
2.0% completed 
7-13
--------------
3.0% completed 
9-21
--------------
4.0% completed 
9-31
--------------
5.0% completed 
11-39
--------------
6.0% completed 
14-46
--------------
7.000000000000001% completed 
16-54
--------------
8.0% completed 
20-60
--------------
9.0% completed 
22-68
--------------
10.0% completed 
27-73
--------------
11.0% completed 
31-79
--------------
12.0% completed 
34-86
--------------
13.0% completed 
35-95
--------------
14.000000000000002% completed 
36-104
--------------
15.0% completed 
38-112
--------------
16.0% completed 
41-119
--------------
17.0% completed 
46-124
--------------
18.0% completed 
49-131
--------------
19.0% completed 
53-137
--------------
20.0% completed 
54-146
--------------
21.0% completed 
57-153
--------------
22.0% completed 
61-159
--------------
23.0% completed 
65-165
--------------
24.0% completed 
70-170
--------------
25.0% completed 
72-178
--------------
26.0% completed 
75-185


In [15]:
println(j)
println(k)
println(j/(j+k))
println(j+k)

280
718
0.280561122244489
998


In [16]:
## New Data Set
Data=CSV.read("C:\\Users\\Abraham\\Dropbox (MIT)\\New folder\\auto-mpg.csv",header=true);
Data=Data[:,[1,2,3,5,6,7,8]];

In [17]:
j=0;
k=0;
iterations=1000;

Auto_Results=zeros(iterations,2)

for i = 1:iterations
    comp = @suppress SVB(Data,0,1,10,false,rand(1:2000019),0.25,false);
    
    Auto_Results[i,1]=comp[1];
    Auto_Results[i,2]=comp[2];
    
    if comp[1]<comp[2]
        j=j+1;
    end
    if comp[2]<comp[1]
        k=k+1;
    end
    if (i%10==0)
        print((i/iterations)*100);
        println("% completed ");
        print(j);
        print("-");
        println(k);
        println("--------------");
    end
end

1.0% completed 
5-5
--------------
2.0% completed 
11-9
--------------
3.0% completed 
20-10
--------------
4.0% completed 
26-14
--------------
5.0% completed 
33-17
--------------
6.0% completed 
38-22
--------------
7.000000000000001% completed 
45-25
--------------
8.0% completed 
50-30
--------------
9.0% completed 
54-36
--------------
10.0% completed 
60-40
--------------
11.0% completed 
68-42
--------------
12.0% completed 
76-44
--------------
13.0% completed 
84-46
--------------
14.000000000000002% completed 
90-50
--------------
15.0% completed 
95-55
--------------
16.0% completed 
101-59
--------------
17.0% completed 
107-63
--------------
18.0% completed 
114-66
--------------
19.0% completed 
121-69
--------------
20.0% completed 
125-75
--------------
21.0% completed 
131-79
--------------
22.0% completed 
136-84
--------------
23.0% completed 
141-89
--------------
24.0% completed 
146-94
--------------
25.0% completed 
153-97
--------------
26.0% completed 
159-101

In [18]:
println(j)
println(k)
println(j/(j+k))
println(j+k)

604
396
0.604
1000


In [19]:
## New Data Set
Data=CSV.read("C:\\Users\\Abraham\\Dropbox (MIT)\\New folder\\abalone.csv",header=true);
Data=Data[:,2:9];

In [20]:
j=0;
k=0;
iterations=1000;

Abalone_Results=zeros(iterations,2)

for i = 1:iterations
    comp = @suppress SVB(Data,0,1,10,false);
    
    Abalone_Results[i,1]=comp[1];
    Abalone_Results[i,2]=comp[2];
    
    if comp[1]<comp[2]
        j=j+1;
    end
    if comp[2]<comp[1]
        k=k+1;
    end
    if (i%10==0)
        print((i/iterations)*100);
        println("% completed ");
        print(j);
        print("-");
        println(k);
        println("--------------");
    end
end

1.0% completed 
10-0
--------------
2.0% completed 
18-2
--------------
3.0% completed 
27-3
--------------
4.0% completed 
37-3
--------------
5.0% completed 
44-6
--------------
6.0% completed 
53-7
--------------
7.000000000000001% completed 
63-7
--------------
8.0% completed 
72-8
--------------
9.0% completed 
82-8
--------------
10.0% completed 
92-8
--------------
11.0% completed 
101-9
--------------
12.0% completed 
111-9
--------------
13.0% completed 
121-9
--------------
14.000000000000002% completed 
131-9
--------------
15.0% completed 
141-9
--------------
16.0% completed 
149-11
--------------
17.0% completed 
158-12
--------------
18.0% completed 
168-12
--------------
19.0% completed 
177-13
--------------
20.0% completed 
185-15
--------------
21.0% completed 
193-17
--------------
22.0% completed 
203-17
--------------
23.0% completed 
212-18
--------------
24.0% completed 
221-19
--------------
25.0% completed 
231-19
--------------
26.0% completed 
241-19
-------

In [21]:
println(j)
println(k)
println(j/(j+k))
println(j+k)

954
46
0.954
1000


In [22]:
#CSV.write("C:\\Users\\Abraham\\Dropbox (MIT)\\New folder\\Full2.csv",DataFrame(hcat(Concrete_Results,ForestFire_Results,Housing_Results,CompHard_Results,Auto_Results,Abalone_Results)),writeheader=false);